In [ ]:
### import libraries

# for DL modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

# for number-crunching
import numpy as np

# for dataset management
import pandas as pd

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

In [ ]:
# import the data
url  = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
data = pd.read_csv(url,sep=',',header=None)
data.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','DISEASE']


# data contain some ?'s; replace with NaN and drop those rows
data = data.replace('?',np.nan).dropna()

data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,DISEASE
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3


In [ ]:
def binarization(num):
  if(num>0):
    return 1
  else:
    return 0

In [ ]:
data['DISEASE'] = data['DISEASE'].apply(binarization)

In [ ]:
Features = data.drop('DISEASE',axis=1)
Values = data['DISEASE']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Features,Values, test_size=0.1, random_state=42)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train = torch.Tensor(X_train).float()
X_test = torch.Tensor(X_test).float()

y_train = torch.Tensor(y_train.values).long()
y_test =  torch.Tensor(y_test.values).long()


In [ ]:
y_train.dtype

torch.int64

In [ ]:
y_test = y_test[:,None]
y_train = y_train[:,None]

In [ ]:
y_test.shape

torch.Size([30, 1])

In [ ]:
TrainDataset = TensorDataset(X_train,y_train)
TestDataset = TensorDataset(X_test,y_test)

TrainLoader =DataLoader(TrainDataset,batch_size=TrainDataset.tensors[0].shape[0],shuffle=True)
TestLoader = DataLoader(TestDataset,batch_size=TestDataset.tensors[0].shape[0])


In [ ]:
class Heart(nn.Module):

  def __init__(self):
    super().__init__()

    self.input = nn.Linear(13,64)
    self.hidden = nn.Linear(64,128)
    self.output = nn.Linear(128,1)

  def forward(self,x):

    x = F.relu(self.input(x))

    x = F.relu(self.hidden(x))

    return (self.output(x))


net = Heart()
loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)


In [ ]:
for epoch in range(50):

  X,y = next(iter(TrainLoader))

  yHat = net(X)

  lossfn = loss(yHat,y.float())

  optimizer.zero_grad()
  lossfn.backward()
  optimizer.step()

  print("Epoch",epoch,"Loss",lossfn.item())

X,y = next(iter(TestLoader))
yHat = net(X)
lossfn = loss(yHat,y.float())
print(lossfn.item())
predictions = (torch.sigmoid(yHat)>.5).float()
100*torch.mean((predictions==y).float())

Epoch 0 Loss 0.6889474987983704
Epoch 1 Loss 0.5901162028312683
Epoch 2 Loss 0.4995776116847992
Epoch 3 Loss 0.41822269558906555
Epoch 4 Loss 0.3679490089416504
Epoch 5 Loss 0.35593974590301514
Epoch 6 Loss 0.3603699207305908
Epoch 7 Loss 0.3578187823295593
Epoch 8 Loss 0.3438856303691864
Epoch 9 Loss 0.3239087760448456
Epoch 10 Loss 0.3040497303009033
Epoch 11 Loss 0.28866472840309143
Epoch 12 Loss 0.27785369753837585
Epoch 13 Loss 0.26889532804489136
Epoch 14 Loss 0.2595486342906952
Epoch 15 Loss 0.2496720254421234
Epoch 16 Loss 0.23999711871147156
Epoch 17 Loss 0.23110802471637726
Epoch 18 Loss 0.22231543064117432
Epoch 19 Loss 0.21198269724845886
Epoch 20 Loss 0.19977426528930664
Epoch 21 Loss 0.187091663479805
Epoch 22 Loss 0.1756439357995987
Epoch 23 Loss 0.16551347076892853
Epoch 24 Loss 0.15524880588054657
Epoch 25 Loss 0.14422494173049927
Epoch 26 Loss 0.1324888914823532
Epoch 27 Loss 0.12101094424724579
Epoch 28 Loss 0.11010889708995819
Epoch 29 Loss 0.09918497502803802
Epoch

tensor(90.)

In [ ]:
TestDataset.tensors[0].shape

torch.Size([30, 13])